# Custom Correlation Functions

In [ ]:
#%matplotlib notebook
import forktps as ftps
import matplotlib.pyplot as plt

import numpy as np

from forktps.DiscreteBath import DiscretizeBath, CalcN, FourierTransform
from forktps.Helpers import *

from triqs.operators import *

In this short tutorial we learn how to compute general custom correlation functions $\langle A(t) B \rangle$ by calculating dynamic spin-spin correlation functions $\langle S^\alpha(t) S^\alpha \rangle$, $\alpha \in \{x,y,z\}$. We compare this to the exact solution for the $\alpha = z$ component and also to the $\alpha = z$ result calculated from the ForkTPS density-density correlator using: $S^z = 0.5( n_{\uparrow} - n_{\downarrow} )$:
$$\langle S^z_i(t) S^z_j\rangle = \frac{1}{4} \bigg( \langle n_{i\uparrow}(t) n_{j\uparrow}\rangle + \langle n_{i\downarrow}(t) n_{j\downarrow}\rangle - \langle n_{i\uparrow}(t) n_{j\downarrow}\rangle - \langle n_{i\downarrow}(t) n_{j\uparrow}\rangle  \bigg).$$

Before we start a couple of comments: <br>
1. ForkTPS allows to calculate the correlation function of arbitrary impurity operators $A$ and $B$. It does that by decomposing $A$ and $B$ into their monomials (string of creation and annihilation operators) and time evolving each term separately. Hence for very complicated operators this could yield a large number of time evolutions needed to do, so be careful. It is guaranteed though that the solver never calculates anything twice. So if two Green's functions need the same time evolution it is computed only once and both can then access it. <br>
2. Currently only positive times $t>0$ are implemented because FTPS is based on the retarded Green's function with the $\Theta(t)$-term in front. This can easily be changed though in future updates. <br><br>
To start, let's initialize a simple impurity model using a semi-circular hybridization and turn off interactions such that we can compare to the exact solution.

In [ ]:
Norbs = 2
Nb = 9 #number of bath sites

inds = list(range(Norbs))
gfstruct = [['up', inds], ['dn', inds]]


# hybridization and discrete bath
dup = GfReFreq(indices = inds, window = (-3., 3.), n_points =  3001, name = "up")
ddn = GfReFreq(indices = inds, window = (-3., 3.), n_points =  3001, name = "dn")

for i in range(Norbs):
    dup[i,i] << SemiCircular(half_bandwidth = 2.0)
    ddn[i,i] << SemiCircular(half_bandwidth = 2.0)

Delta = BlockGf(name_list = ('up','dn'), block_list = (dup,ddn), make_copies = False)
bath = DiscretizeBath(Delta=Delta, Nb=Nb) #bath object storing all relevant information for the bath

# non-interacting impurity hamiltonian
e0 = ftps.solver_core.Hloc(gfstruct) #give the local Hamiltonian the right block structure
e0.Fill( [[0,0],[0,0]] )

# Interaction parameters 
Hint = ftps.solver_core.HInt(u=0.)

S = ftps.Solver(gf_struct = gfstruct , nw = 3001, wmin=-3., wmax=3.)
S.b = bath
S.e0 = e0

The solver differentiates between two types of Green's functions. Default ones including most prominently the single-particle Green's function and the Density-Density correlation function and so-called custom Green's functions for arbitrary operators $A$ and $B$.

To calculate certain default Green's functions one should use the parameter "GFs" in the ```S.solve()``` function. It is a list of strings specifying which  default Green's function the solver should compute:

```Python
GFs = ['S'] # Single-Particle Green's function only
GFs = ['N'] # Density-Density Green's function
GFs = ['S','N'] # Single-Particle and Density-Density Green's function
``` 

Custom Green's functions can be defined using the parameter customGF. This argument is a list of pairs of TRIQS-operators:

```Python
Sz = ... #define operator Sz
Sx = ... #define operator Sx

S.solve( ...
         customGF = [[Sz,Sz], # calculate <Sz(t) Sz>
                     [Sx,Sx]],# calculate <Sx(t) Sz>
         ... )

``` 

Of course one could also use the customGF argument to calculate single-particle Green's functions, but they will end up in a different output containers (see below).

So let us calculate a couple of Green's functions:

In [ ]:
# Time evolution parameters
tevoParams = ftps.solver.TevoParams(dt = 0.1, time_steps=30)

# define spin-operators only for orbital 0
Sz = 0.5* ( n('up',0) - n('dn',0) )   
Sx = 0.5* ( c_dag('up',0)*c('dn',0) + c_dag('dn',0)*c('up',0) )
Sy = 0.5j*( c_dag('up',0)*c('dn',0) - c_dag('dn',0)*c('up',0)  )

# Solve the impurity problem for density-density GF
S.solve( h_int = Hint, tevo = tevoParams,
         GFs= ['N'],                # density-density only ie no single particle G
         customGF = [[Sx,Sx],       # spin-spin correlators, for the spin-symmetric model defined above we expect them all to be equal
                     [Sy,Sy],
                     [Sz,Sz]]
       )

We can access the default density-density Green's function using the ```S.N_t``` member. On the other hand, custom Green's functions can be obtained from the ```S.customGF``` member which is a list of scalar TRIQS Green's functions. 
For the non-interacting problem studied in this tutorial we can also calculate the exact density-density Green's function using the function ```CalcN``` from ```DiscreteBath.py```. So let us compare the different Green's functions starting with the density-density one because for it we have the exact solution:

In [ ]:
NNCorr = S.N_t
NNCorrExact = CalcN(bath=bath, hloc=e0, mesh= NNCorr.mesh)

shape = NNCorr.target_shape

plt.figure(figsize = (8, 4))
plt.suptitle('Density-Density Correlation function.')
for indx, [i,j] in enumerate( product( range(shape[0]), range(shape[1]) ) ):
    plt.subplot(shape[0], shape[1], indx+1)
    
    x = getX(NNCorr.mesh)
    plt.plot( x, NNCorr[i,j].real.data , 'x-', markevery=5, label = 'FTPS' )
    plt.plot( x, NNCorrExact[i,j].real.data, label = 'Exact' )
    plt.ylim(0,0.6)
    plt.xlim(0,6)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    
    if indx==1:
        plt.legend()

    
plt.figure(figsize = (8, 4))
plt.suptitle(R'Numerical error: $| \langle n(t)n\rangle_{exact} - \langle n(t)n\rangle_{FTPS} |$')
for indx, [i,j] in enumerate( product( range(shape[0]), range(shape[1]) ) ):
    plt.subplot(shape[0], shape[1], indx+1)
    y = np.abs(NNCorr[i,j].real.data -NNCorrExact[i,j].real.data)
    plt.semilogy( x, y )
    plt.ylim(1E-8,5E-5)
    plt.xlim(0,6)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    

The first thing we note is that the density-density correlation does not respect the block structure as the single particle Green's function would. Instead it has entries connecting the up-block with the down block which in our trivial non-interacting example are constant, but become dynamic as soon as we would switch on interactions. 
<br> 
Hence the order of the entries in the density-density Green's functions is important to know. It is given is:
```
0 ... Block 'up' index 0
1 ... Block 'dn' index 0
2 ... Block 'up' index 1
3 ... Block 'dn' index 1
           ...
```
Such that for example ```NNCorr[1,2]``` represents the correlation function $\langle n_{0\downarrow}(t) n_{1,\uparrow} \rangle$.

Looking at the actual results we find that indeed the agreement with the exact result is very good with errors of the order of $10^{-5}$. 

Finally let us calculate the SzSz correlation function and compare it to the custum Green's functions we also computed:

In [ ]:
def SzSz(NN, orbI, orbJ):
    # calculates spin-spin correlation between orbI and orbJ
    
    #indices off NN
    upI = 2*orbI
    upJ = 2*orbJ
    dnI = upI + 1
    dnJ = upJ + 1
    
    SzSz = NN[upI,upJ] + NN[dnI,dnJ] - NN[upI, dnJ] - NN[dnI, upJ]
    return 0.25*SzSz

plt.figure()
SzSzCorr = SzSz(NNCorr, 0, 0)
plt.plot( getX(SzSzCorr.mesh), SzSzCorr.real.data, label = R'$\langle S^z(t) S^z \rangle$ using $\langle N(t)N \rangle$' )
names = [R'$\langle S^x(t) S^x \rangle$', R'$\langle S^y(t) S^y \rangle$', R'$\langle S^z(t) S^z \rangle$'] 
for gf, name in zip(S.customGF, names):
    plt.plot( getX(gf.mesh), gf.real.data, label = name + ' using customGF' )
plt.legend()



We find that they all agree but if one zooms in one finds that $\langle S^x(t)S^x\rangle$ and $\langle S^y(t)S^y\rangle$ are exactly the same and so are the two versions of $\langle S^z(t)S^z\rangle$. This is because the x-spin and y-spin correlations are actually calculated form the exact same time evolved states. The same is true for the z-spin and the density-density terms.